In [1]:
import cv2
import numpy as np

In [3]:
def show_images(imgList):
    while True:
        for (name,img) in imgList:
            cv2.imshow(name,img)
        pressed_key = cv2.waitKey(0)
        if pressed_key==27:
            break
    cv2.destroyAllWindows()

In [73]:
def inhomogenize(p):
    return p/p[2]

In [77]:
def get_affine(p1,p2,p3,p4):
    ## p1 p2 p3 p4 in anticlockwise order of square
    p1,p2,p3,p4 = np.concatenate(([p1,p2,p3,p4],np.ones([4,1],dtype=np.float32)),axis=1)
    
    l1 = inhomogenize(np.cross(p1,p2))
    l2 = inhomogenize(np.cross(p3,p4))
    l3 = inhomogenize(np.cross(p2,p3))
    l4 = inhomogenize(np.cross(p1,p4))
    
    a = inhomogenize(np.cross(l1,l2))
    b = inhomogenize(np.cross(l3,l4))
    
    linf = inhomogenize(np.cross(a,b))
    
    return np.array([[1,0,0],[0,1,0],[linf[0],linf[1],1]],dtype=np.float32)

In [87]:
def get_metric(p1,p2,p3,p4):
    ## p1 p2 p3 p4 in anticlockwise order of square
    img_points = np.array([p1,p2,p3,p4],dtype=np.float32)
    size = (np.linalg.norm(p1-p2)
            +np.linalg.norm(p2-p3)
            +np.linalg.norm(p3-p4)
            +np.linalg.norm(p4-p1))/4
    cube_points = np.array([[p1[0],p1[1]],
                            [p1[0],p1[1]+size],
                            [p1[0]+size,p1[1]+size],
                            [p1[0]+size,p1[1]]],dtype=np.float32)
    return cv2.getPerspectiveTransform(points,cube_points)

## Input image

In [2]:
IMG = cv2.imread("./images/persp_tile.jpg")

In [27]:
show_images([("Img",IMG)])

In [95]:
# points = np.array([[1458,194],[1965,145],[1880,754],[1400,687]],dtype=np.float32)
points = np.array([[194,1458],[145,1965],[754,1880],[687,1400]],dtype=np.float32)

In [88]:
H_met = get_metric(*points)

In [89]:
H_aff = get_affine(*points)

In [90]:
met_img = cv2.warpPerspective(IMG,H_met,(IMG.shape[1],IMG.shape[0]))
aff_img = cv2.warpPerspective(IMG,H_aff,(IMG.shape[1],IMG.shape[0]))

In [96]:
p1,p2,p3,p4 = points

526.32977294921875

In [ ]:
a

In [91]:
show_images([("Orig",IMG),("Metric",met_img),("Affine",aff_img)])